In [4]:
import numpy as np
# 행렬을 입력하기 위해 np.array를사용한다 
x= np.array([[1,2],
            [3,4]])
# 1은 0,1의행
#행렬곱은 교환법칙이 성립되지않음
x

array([[1, 2],
       [3, 4]])

In [5]:
x[0]

array([1, 2])

In [6]:
x[1]

array([3, 4])

In [9]:
y = np.array([[5,6],[7,8]])
y

array([[5, 6],
       [7, 8]])

In [11]:
x*y==y*x # 성립하는경우도 있음


array([[ True,  True],
       [ True,  True]])

In [14]:
def sigmoid(x):
    return 1 / (1 +np.exp(-x))

def identity_function(x):
    return x

def init_network(): # 모델의 틀을 미리 만듬  보통 이렇게 한다..
    network= {} # 딕셔너리 
    network['W1'] = np.array([[0.1, 0.3, 0.5], [0.2, 0.4, 0.6]])
    network['b1'] = np.array([[0.1, 0.2, 0.3]])
    network['W2'] = np.array([[0.1, 0.4],[0.2, 0.5],[0.3, 0.6]])
    network['b2'] = np.array([[0.1, 0.2]])
    network['W3'] = np.array([[0.1, 0.3],[0.2, 0.4]])
    network['b3'] = np.array([[0.1, 0.2]])

    return network

def forward(network , x):  #순전파의 역할을 함
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x,W1)+ b1  # 행렬 곱은 np.dot을 쓴다. # 입력층에서 1층 은닉층으로 
    z1 = sigmoid(a1) # a1에서 가중치를 재조정 해준다
    a2 = np.dot(z1,W2) + b2  # 1층에서 2층으로
    z2 = sigmoid(a2) # 가중치를 재조정
    a3 = np.dot(z2,W3) + b3 # 2층에서 3층으로 
    y = identity_function(a3)
    # 항등함수 y는 사실 a3랑 같음 -> 일관성을 위해 넣음 -> 이후엔 softmax함수를 사용한다.
    
network = init_network() # 모든 인공신경망을 담고 있다. 모델을 정의함
x = np.array([1.0,0.5])
y = forward(network, x)
print(y)

None


# softmax 적용 하는 방법 
e^x을 적용해 양수로 변경->exp 연산 

그이후 노말라이즈를 적용 -> 원소를 다 더한 값으로 나눔

## np.exp(값)/np.sum(np.exp(값))

그러면 값이 큰 경우 컴퓨터가 연산을 못함 -> overflow가 나올 확률이 나옴 

그렇기에 동일한 숫자를 더해도 동일한 값이 나온다는것을 적용해 값을 

낮추면 overflow가 나오지 않을수 있다.


In [15]:
def relu(x):
    return np.maximum(0, x)

def relu_grad(x):
    grad = np.zeros(x)
    grad[x>=0] = 1
    return grad

def softmax(x):
    if x.ndim == 2: # ndim은 numpy로 말한다. 배치 처리시 사용
        x = x.T
        x = x - np.max(x,axis=0)
        y = np.exp(x) / np.sum(np.exp(x),axis = 0)
        return y.T

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x)/np.sum(np.exp(x))

def mean_squared_error(y,t):
    return 0.5 * np.sum((y-t)**2)
def cross_entropy_error(y,t):
    if y.ndim == 1: # 차원을 묻는거임 
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    # 훈련데이터가 원 - 핫 벡터라면 정답 레이블의 인덱스로 변환 
    if t.size == y.size:
        t = t.argmax(axis=1)
        batch_size = y.shape[0]
    return - np.sum(np.log(y[np.arrange(batch_size),t])) / batch_size
    
def softmax_loss(X,t):
    y = softmax(X)
    return cross_entropy_error(y, t)




3층 신경망의 신호 전달

 # 기본적인 구조
##### Affine -> Sigmoid -> Affine -> Sigmoid -> Affine -> Softmax
Affine은 Wx+b를 연산한거 